In [2]:
from pynq import Overlay, allocate
import numpy as np

# Load the bitstream (assuming the name of the bitstream is 'dma_echo.bit')
overlay = Overlay("buf.bit")

# Get the DMA instance
dma = overlay.axi_dma_0  # Adjust this based on your Vivado project name for the DMA

# Number of integers to send (change this to test different lengths)
num_integers = 25

# Allocate memory for input and output buffers
# 'allocate' method allocates physically contiguous memory for use by the DMA
input_buffer = allocate(shape=(num_integers,), dtype=np.uint32)
output_buffer = allocate(shape=(num_integers,), dtype=np.uint32)

# Fill the input buffer with some integers to send to the IP (e.g., 1, 2, 3, ...)
for i in range(num_integers):
    input_buffer[i] = i + 1

# Initialize the output buffer with zeros (optional, just for clarity)
output_buffer[:] = np.zeros(num_integers, dtype=np.uint32)

print("Input Buffer: ", input_buffer)

# Start the DMA send (from PS to PL)
dma.sendchannel.transfer(input_buffer)

# Start the DMA receive (from PL to PS)
dma.recvchannel.transfer(output_buffer)

# Wait for both send and receive transfers to complete
dma.sendchannel.wait()
dma.recvchannel.wait()

print("Output Buffer (Data echoed from the IP): ", output_buffer)

# Verify if the output matches the input
if np.array_equal(input_buffer, output_buffer):
    print("Test Passed! The output matches the input.")
else:
    print("Test Failed! The output does not match the input.")

# Clean up the buffers
input_buffer.close()
output_buffer.close()


Input Buffer:  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25]
Output Buffer (Data echoed from the IP):  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25]
Test Passed! The output matches the input.
